# Meta Inc. Stock price predictions

Let's take a look to stock prices and market capitalisation as a result of how market reacts to financial statistics of the company. 
First of all we need to extract financial statistics from the web. After that we need to determine affection of this statistics to stock price and discover correlation.
Final step will be building model to predict market capitalisation for the next quorter.

Import libraries

In [1]:
import pandas as pd
!mamba install bs4==4.10.0 -y
!mamba install html5lib==1.1 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.7.6) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['bs4==4.10.0']

pkgs/main/linux-64       [>                   ] (--:--) No change
pkgs/main/linux-6

In [2]:
#Import libraries for webscrapping
import requests
from bs4 import BeautifulSoup

Set url and get request

In [3]:
url = "https://www.macrotrends.net/stocks/charts/META/meta-platforms/revenue"

html_data = requests.get(url).text   #get data from the webpage
souped_html = BeautifulSoup(html_data, 'html5lib') #Use beautifulsoup
souped_html.title

<title>Meta Platforms Revenue 2010-2022 | META | MacroTrends</title>

In [4]:
# Find the second table with Quarterly Revenue /or use 0 for first table
revenue_table = souped_html.find_all('table')[1]   

#Create blank dataframe 
revenue_dataframe = pd.DataFrame(columns=["Date", "Revenue"]) 
 
# Find all tr tags and put data into Dataframe
count_of_missed_rows = 0
for row in revenue_table.find("tbody").find_all("tr"):  

    col = row.find_all("td") 
    date = col[0].text 
    Revenue = col[1].text
    Revenue = Revenue.replace('$', '').replace(',', '')

    if Revenue.isdigit():
        Revenue = float(Revenue)
        revenue_dataframe = revenue_dataframe.append({"Date":date, "Revenue":Revenue}, ignore_index=True)
    else: 
        count_of_missed_rows = count_of_missed_rows + 1
        
print("Missed rows:", count_of_missed_rows) #How many rows were missed

Missed rows: 1


In [5]:
market_cap_url = "https://www.marketcaphistory.com/meta-platforms/"

In [9]:
html_data_mc = requests.get(market_cap_url).text   #get data from the webpage
souped_html_mc = BeautifulSoup(html_data_mc, 'html5lib') #Use beautifulsoup
souped_html_mc.title

<title>Meta Platforms Market Cap History</title>

In [10]:
# Find the second table with Quarterly Revenue /or use 0 for first table
mc_table = souped_html_mc.find_all('table')  

#Create blank dataframe 
mc_dataframe = pd.DataFrame(columns=["Date", "Market Capitalisation"])

In [12]:
mc_table[0]

<table cellpadding="0" cellspacing="0" class="infotable">
<tbody><tr><td class="menu-item">FB Market Cap History Table</td></tr>
<tr><td class="infotablebox"><font face="Arial" size="2">Below is a table of the <strong>Meta Platforms market cap history</strong> going back to 5/18/2012:</font><p>
</p><table cellpadding="4" cellspacing="0" style="margin-top: 20px" width="290">
<tbody><tr><td align="center" style="padding: 6px; background-color: #EEEEEE; border-bottom: 1px solid #CCCCCC; font-size: 13px;" width="40%"><i><b>Date</b></i></td>
<td align="center" style="padding: 6px; background-color: #EEEEEE; border-bottom: 1px solid #CCCCCC; font-size: 13px;" width="60%"><i><b>FB Market Cap</b></i></td></tr>
<tr><td align="center" class="tstyle"><i>5/18/2012</i></td><td align="center" class="tstyle">24.31B</td></tr><tr><td align="center" class="tstyle"><i>7/25/2012</i></td><td align="center" class="tstyle">19.79B</td></tr><tr><td align="center" class="tstyle"><i>10/22/2012</i></td><td align=